In [25]:
import pyspark

myConf = pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
        .builder\
        .master('local')\
        .config(conf=myConf)\
        .appName('dataframe_prac')\
        .getOrCreate()

21/10/02 20:10:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
myList=[('1','kim, js', 170),
        ('1','lee, sm', 175),
        ('2','lim, yg',180),
        ('2','lee', 170)]
myDf = spark.createDataFrame(myList)

In [4]:
myDf.columns

['_1', '_2', '_3']

In [5]:
myDf.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: long (nullable = true)



In [6]:
myDf.take(1)

[Row(_1='1', _2='kim, js', _3=170)]

In [7]:
from pyspark.sql import Row

Person = Row("Year", "Name", "Height")
myRows = [
    Person("1", "Lee", 175),
    Person("2", "Lim", 180),
    Person("2", "Kim", 170)
]
myDf = spark.createDataFrame(myRows)

In [8]:
myDf.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Height: long (nullable = true)



In [9]:
myDf.show()

+----+----+------+
|Year|Name|Height|
+----+----+------+
|   1| Lee|   175|
|   2| Lim|   180|
|   2| Kim|   170|
+----+----+------+



In [10]:
myDf.toPandas()

,Year,Name,Height
0,1,Lee,175
1,2,Lim,180
2,2,Kim,170


In [11]:
myDf.write.format('com.databricks.spark.csv').save("data/_myDf.csv")

In [19]:
!ls data/_myDf.csv
!cat data/_myDf.csv/part-00000-d3e5d318-6aac-4cb2-82e7-6e180bd5f211-c000.csv

_SUCCESS
part-00000-d3e5d318-6aac-4cb2-82e7-6e180bd5f211-c000.csv
1,Lee,175
2,Lim,180
2,Kim,170


In [22]:
myDf.toPandas().to_csv('data/myDf.csv')

In [23]:
!cat data/myDf.csv

,Year,Name,Height
0,1,Lee,175
1,2,Lim,180
2,2,Kim,170


In [24]:
import pandas as pd
icc = pd.DataFrame({"Country": ["South Koread", "Japan", "HongKong"],
                   "codes": [81, 82, 852]})
icc

,Country,codes
0,South Koread,81
1,Japan,82
2,HongKong,852


In [25]:
icc[icc["codes"] == 81]

,Country,codes
0,South Koread,81


In [27]:
from pyspark.sql import Row
lines = spark.sparkContext.textFile("data/ds_spark_2cols.csv")

In [29]:
coll2 = lines.map(lambda l: l.split(","))\
            .map(lambda p: Row(col1=int(p[0].strip()), 
                               col2=int(p[1].strip())))
myDf = spark.createDataFrame(coll2)
myDf.collect()

[Row(col1=35, col2=2),
 Row(col1=40, col2=27),
 Row(col1=12, col2=38),
 Row(col1=15, col2=31),
 Row(col1=21, col2=1),
 Row(col1=14, col2=19),
 Row(col1=46, col2=1),
 Row(col1=10, col2=34),
 Row(col1=28, col2=3),
 Row(col1=48, col2=1),
 Row(col1=16, col2=2),
 Row(col1=30, col2=3),
 Row(col1=32, col2=2),
 Row(col1=48, col2=1),
 Row(col1=31, col2=2),
 Row(col1=22, col2=1),
 Row(col1=12, col2=3),
 Row(col1=39, col2=29),
 Row(col1=19, col2=37),
 Row(col1=25, col2=2)]

In [30]:
%%writefile data/ds_spark.csv
1,2,3,4
11,22,33,44
111,222,333,444

Writing data/ds_spark.csv


In [31]:
df = spark.read\
    .format('com.databricks.spark.csv')\
    .options(header='true', inferschema='true', delimiter=",")\
    .load("data/ds_spark.csv")
df.show()

+---+---+---+---+
|  1|  2|  3|  4|
+---+---+---+---+
| 11| 22| 33| 44|
|111|222|333|444|
+---+---+---+---+



In [34]:
df = spark.read\
    .options(header='true', inferschma='true', delimiter=',')\
    .csv('data/ds_spark.csv')
df.show()

+---+---+---+---+
|  1|  2|  3|  4|
+---+---+---+---+
| 11| 22| 33| 44|
|111|222|333|444|
+---+---+---+---+



In [8]:
import json

with open('data/ds_twiiter_seoul_3.json') as f:
    data = f.readlines()

type(data)

list

In [9]:
len(data)

1

In [13]:
jsonStr = json.loads(data[0])
jsonStr

{'contributors': None,
 'truncated': False,
 'text': 'RT @soompi: #SEVENTEEN’s Mingyu, Jin Se Yeon, And Leeteuk To MC For 2016 Super Seoul Dream Concert \nhttps://t.co/1XRSaRBbE0 https://t.co/fi…',
 'is_quote_status': False,
 'in_reply_to_status_id': None,
 'id': 801657325836763136,
 'favorite_count': 0,
 'entities': {'symbols': [],
  'user_mentions': [{'id': 17659206,
    'indices': [3, 10],
    'id_str': '17659206',
    'screen_name': 'soompi',
    'name': 'Soompi'}],
  'hashtags': [{'indices': [12, 22], 'text': 'SEVENTEEN'}],
  'urls': [{'url': 'https://t.co/1XRSaRBbE0',
    'indices': [100, 123],
    'expanded_url': 'http://www.soompi.com/2016/11/20/seventeens-mingyu-jin-se-yeon-leeteuk-mc-dream-concert/',
    'display_url': 'soompi.com/2016/11/20/sev…'}]},
 'retweeted': False,
 'coordinates': None,
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'in_reply_to_screen_name': None,
 'in_reply_to_user_id': None,
 'retweet_count': 1487

In [20]:
type(jsonStr)
jsonStr = b'[' + b','.join(data) + b"["

TypeError: sequence item 0: expected a bytes-like object, str found

In [22]:
import pandas as pd

tweetPd = pd.read_json('data/ds_twiiter_seoul_3.json')
tweetPd.shape

(68, 26)

In [23]:
tweetPd["id"][:10]

symbols                  801657325836763136
user_mentions            801657325836763136
hashtags                 801657325836763136
urls                     801657325836763136
contributors             801657325836763136
truncated                801657325836763136
text                     801657325836763136
is_quote_status          801657325836763136
in_reply_to_status_id    801657325836763136
id                       801657325836763136
Name: id, dtype: int64

In [26]:
tweetDf = spark.read.json('data/ds_twiiter_seoul_3.json')
tweetDf.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- symbols: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- urls: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- url: string (nullable = true)
 |    |-- user_mentions: array (nullable = true)
 |    |    |-- element: struct (containsNull 

21/10/02 20:10:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
